<a href="https://colab.research.google.com/github/osgeokr/kari-sdm/blob/main/002_gbif_pitta_nympha_spatial_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-foss4g')

In [2]:
import geemap
import os, requests
import pandas as pd, geopandas as gpd
import matplotlib.pyplot as plt

from google.colab import files

In [3]:
# 대한민국 범위 좌표 지정(좌하단 : 우상단 좌표)
rectangle_coords = [
    [125.0765578311700068, 33.1124998462386984],
    [131.8727812628719960, 38.4000004985049017],
]

# 직사각형 생성
rectangle = ee.Geometry.Rectangle(rectangle_coords)

# 직사각형을 Feature로 변환
feature = ee.Feature(rectangle)

# FeatureCollection 생성
kr = ee.FeatureCollection([feature])

m = geemap.Map(layout={'height':'400px', 'width':'800px'})
m.addLayer(kr, {}, "Republic of Korea")
m.centerObject(kr, 6)
m

Map(center=[35.77459004930315, 128.47466954702105], controls=(WidgetControl(options=['position', 'transparent_…

### Dynanic World

2019년 2023년 중 5월부터 7월까지 기간의 이미지만 필터링한 후, 가장 빈번하게 나타나는 label 값, 즉 최다 빈도(most frequent)를 각 픽셀에 대해 계산하였다.

In [4]:
# 날짜 범위 설정
start_date = '2019-05-01'
end_date = '2023-07-31'

# Dynamic World 이미지 컬렉션 필터링
dw = (ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
      .filterDate(start_date, end_date)
      .filterBounds(kr.geometry()))

In [5]:
# ImageCollection의 이미지 개수
layer_count = dw.size().getInfo()
print(f'dw의 레이어 수: {layer_count}')

dw의 레이어 수: 15440


In [6]:
# 5월부터 7월까지 이미지 필터링
dw_filtered = dw.filter(ee.Filter.calendarRange(5, 7, 'month'))

# ImageCollection의 이미지 개수
layer_count = dw_filtered.size().getInfo()
print(f'dw의 레이어 수: {layer_count}')

dw의 레이어 수: 3851


In [7]:
# Dynamic World 이미지 컬렉션의 'label' 밴드 선택
classification = dw.select('label')

# 'mode' 리듀서를 사용하여 합성 이미지 생성
dw_composite = classification.reduce(ee.Reducer.mode())

In [11]:
# 시각화 매개변수 설정
dw_vis_params = {
    'min': 0,
    'max': 8,
    'palette': [
        '#419BDF', '#397D49', '#88B053', '#7A87C6', '#E49635',
        '#DFC35A', '#C4281B', '#A59B8F', '#B39FE1'
    ]
}

# 합성 이미지를 지리적 범위로 자르기
dw_composite_clipped = dw_composite.clip(kr.geometry())

m = geemap.Map(layout={'height':'400px', 'width':'800px'})
m.addLayer(dw_composite_clipped, dw_vis_params, 'Classified Composite')
m.centerObject(dw_composite_clipped, 9)
m

Map(center=[35.77459004930315, 128.47466954702105], controls=(WidgetControl(options=['position', 'transparent_…

In [13]:
%%time

import time

# 이미지를 Google Drive에 내보내기
task = ee.batch.Export.image.toDrive(
    image=dw_composite_clipped,
    description='DW_2019-2023_MAY-JUL',
    folder='kari-sdm',
    scale=10,  # 이미지의 해상도
    region=kr.geometry(),  # 내보낼 영역
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)

# 내보내기 작업 시작
task.start()

# 내보내기 작업 상태 확인
print('Export task started. Checking status...')

while True:
    status = task.status()
    state = status['state']
    print('Polling for task (id: {}). Status: {}'.format(task.id, state))
    if state in ['COMPLETED', 'FAILED']:
        break
    time.sleep(30)  # 30초 간격으로 상태 확인

# 완료 후 상태 출력
print('Task completed. Final status:')
print(status)

Export task started. Checking status...
Polling for task (id: UITNS4K6QIDXSZSP53RV2QWM). Status: READY
Polling for task (id: UITNS4K6QIDXSZSP53RV2QWM). Status: RUNNING
Polling for task (id: UITNS4K6QIDXSZSP53RV2QWM). Status: RUNNING
Polling for task (id: UITNS4K6QIDXSZSP53RV2QWM). Status: RUNNING
Polling for task (id: UITNS4K6QIDXSZSP53RV2QWM). Status: RUNNING
Polling for task (id: UITNS4K6QIDXSZSP53RV2QWM). Status: RUNNING
Polling for task (id: UITNS4K6QIDXSZSP53RV2QWM). Status: RUNNING
Polling for task (id: UITNS4K6QIDXSZSP53RV2QWM). Status: RUNNING
Polling for task (id: UITNS4K6QIDXSZSP53RV2QWM). Status: RUNNING
Polling for task (id: UITNS4K6QIDXSZSP53RV2QWM). Status: RUNNING
Polling for task (id: UITNS4K6QIDXSZSP53RV2QWM). Status: RUNNING
Polling for task (id: UITNS4K6QIDXSZSP53RV2QWM). Status: RUNNING
Polling for task (id: UITNS4K6QIDXSZSP53RV2QWM). Status: RUNNING
Polling for task (id: UITNS4K6QIDXSZSP53RV2QWM). Status: RUNNING
Polling for task (id: UITNS4K6QIDXSZSP53RV2QWM). Sta